### PySpark reading a file form a CSV

In [1]:
# import warnings
warnings.filterwarnings("ignore")

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BigData") \
    .config("spark.driver.memory", "16g") \
    .getOrCreate()


24/03/18 10:40:26 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
### Basic - EDA

In [4]:
df_emails = spark.read.format("csv").option("header", "true").load("hdfs://localhost:8020/user1/emails.csv")
df_emails.show()


+--------------------+--------------------+
|                file|             message|
+--------------------+--------------------+
|allen-p/_sent_mai...|Message-ID: <1878...|
|           Date: Mon| 14 May 2001 16:3...|
|From: phillip.all...|                null|
|To: tim.belden@en...|                null|
|           Subject: |                null|
|   Mime-Version: 1.0|                null|
|Content-Type: tex...|                null|
|Content-Transfer-...|                null|
|X-From: Phillip K...|                null|
|X-To: Tim Belden ...|                null|
|              X-cc: |                null|
|             X-bcc: |                null|
|X-Folder: \Philli...| Phillip K.\'Sent...|
|   X-Origin: Allen-P|                null|
|X-FileName: palle...|                null|
|Here is our forecast|                null|
|                   "|                null|
|allen-p/_sent_mai...|Message-ID: <1546...|
|           Date: Fri| 4 May 2001 13:51...|
|From: phillip.all...|          

In [5]:
# Describe provides summary statistics of numeric columns in a DataFrame
df_emails.describe().show()


+-------+--------------------+--------+
|summary|                file| message|
+-------+--------------------+--------+
|  count|             8299853| 2508249|
|   mean|                 NaN|Infinity|
| stddev|                 NaN|     NaN|
|    min|                  \t|      \t|
|    max|~~~~~~~~~~~~~~~~~...|       ||
+-------+--------------------+--------+



In [6]:
# Show the first few rows
df_emails.show(n=20)


+--------------------+--------------------+
|                file|             message|
+--------------------+--------------------+
|allen-p/_sent_mai...|Message-ID: <1878...|
|           Date: Mon| 14 May 2001 16:3...|
|From: phillip.all...|                null|
|To: tim.belden@en...|                null|
|           Subject: |                null|
|   Mime-Version: 1.0|                null|
|Content-Type: tex...|                null|
|Content-Transfer-...|                null|
|X-From: Phillip K...|                null|
|X-To: Tim Belden ...|                null|
|              X-cc: |                null|
|             X-bcc: |                null|
|X-Folder: \Philli...| Phillip K.\'Sent...|
|   X-Origin: Allen-P|                null|
|X-FileName: palle...|                null|
|Here is our forecast|                null|
|                   "|                null|
|allen-p/_sent_mai...|Message-ID: <1546...|
|           Date: Fri| 4 May 2001 13:51...|
|From: phillip.all...|          

In [7]:
from pyspark.sql.functions import regexp_extract

# Example regular expression pattern for a date in the format "E, dd MMM yyyy HH:mm:ss Z"
# Adjust this pattern to match the actual format found in your 'message' data
date_pattern = r'\bMon, \d{2} \w{3} \d{4} \d{2}:\d{2}:\d{2} -\d{4} \(PDT\)'

# Create a new column 'ExtractedDate' by extracting the date string from 'message'
df_emails = df_emails.withColumn("ExtractedDate", regexp_extract("message", date_pattern, 0))

# Show the result of extraction
df_emails.select("ExtractedDate").show(truncate=False, n=5)


+-------------+
|ExtractedDate|
+-------------+
|             |
|             |
|null         |
|null         |
|null         |
+-------------+
only showing top 5 rows



In [8]:
##Preprocessing data

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace

# Assuming SparkSession has already been created
# spark = SparkSession.builder.appName("EmailsAnalysis").getOrCreate()

# Make sure df_emails is properly defined here
# df_emails = spark.read.format("csv").option("header", "true").load("hdfs://localhost:8020/user1/emails.csv")

# Convert the text to lower case
df_emails = df_emails.withColumn("message_clean", lower(col("message")))

# Remove email metadata, non-letter characters, and extra spaces
df_emails = df_emails.withColumn("message_clean", regexp_replace("message_clean", "^(From:|To:|Subject:|Mime-Version:|Content-Type:|Content-Transfer-Encoding:|X-From:|X-To:|X-cc:|X-bcc:|X-Folder:|X-Origin:|X-FileName:).*", ""))
df_emails = df_emails.withColumn("message_clean", regexp_replace("message_clean", "[^a-zA-Z\\s]", ""))
df_emails = df_emails.withColumn("message_clean", regexp_replace("message_clean", "\s+", " "))

# Show the cleaned text
df_emails.select("message_clean").show(truncate=False, n=5)


+----------------------------+
|message_clean               |
+----------------------------+
|messageid javamailevansthyme|
| may pdt                    |
|null                        |
|null                        |
|null                        |
+----------------------------+
only showing top 5 rows



In [10]:
### Step 3: Feature Engineering and Vectorization

In [11]:
from pyspark.sql.functions import col, lower, regexp_replace, to_timestamp
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml import Pipeline

# Ensure df_emails is correctly defined and available at this point in your code

# Filter out rows where 'message_clean' is null or an empty string
df_emails_filtered = df_emails.filter(col("message_clean").isNotNull() & (col("message_clean") != ""))

# Define the stages of the pipeline
tokenizer = Tokenizer(inputCol="message_clean", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
cv = CountVectorizer(inputCol="filtered_words", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="features")

pipeline = Pipeline(stages=[tokenizer, remover, cv, idf])

# Apply the pipeline to the filtered DataFrame
model = pipeline.fit(df_emails_filtered)
result = model.transform(df_emails_filtered)

# Show the result
result.select("features").show(truncate=False)


24/03/18 10:41:41 WARN DAGScheduler: Broadcasting large task binary with size 1803.5 KiB


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                                                                            

24/03/18 10:42:10 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB


##### Your model successfully transformed the textual data into numerical vectors that can be used for machine learning purposes, including neural network models for detecting suspicious messages. The warning about broadcasting a large task binary size is an indication of the data size being processed but is generally not a concern unless it leads to performance issues.

## Step 4: Designing the Neural Network which come  Before moving on to training a neural network model, we'll need to prepare your dataset further, including splitting it into training and test sets, and potentially normalizing the feature vectors 

In [12]:
# Split the data
(train_data, test_data) = result.randomSplit([0.8, 0.2], seed=42)


In [13]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType

# Example UDF to label emails based on the presence of a "suspicious keyword"
def label_email(content):
    if content is not None and "cash" in content:
        return 1
    else:
        return 0

# Register the UDF
label_udf = udf(label_email, IntegerType())

# Assuming 'result' is your DataFrame and 'message_clean' is the column containing the cleaned email text
# Apply the UDF to create a new column 'label'
result = result.withColumn('label', label_udf(col('message_clean')))

# Now proceed with data preparation steps such as splitting the dataset into training and test sets
(train_data, test_data) = result.randomSplit([0.8, 0.2], seed=42)

# Ensure your model training code below is correctly referring to 'features' and 'label' columns
# For example:
from pyspark.ml.classification import LogisticRegression

# Initialize the classifier, assuming 'features' column contains vectorized features
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

# Train the model on the training data
lrModel = lr.fit(train_data)

# Make predictions on the test data
predictions = lrModel.transform(test_data)

# Evaluate the model if necessary
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="label")
auc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

print(f"Test AUC: {auc}")


24/03/18 10:42:11 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:43:13 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:44:11 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:44:12 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:44:12 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:44:13 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:44:13 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:44:14 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:44:14 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:44:15 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:44:15 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/03/18 10:44:16 WARN DAGScheduler: Broadcasting larg

Test AUC: 0.9982873684951179


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Placeholder: Load your data here
# For example, let's assume you've loaded and prepared your datasets into these variables
# X_train, X_test, y_train, y_test = load_and_preprocess_your_data()

# Example placeholder data - replace with your actual data
X_train = np.random.randint(0, 10000, (1000, 100))  # Random data for illustration
y_train = np.random.randint(0, 2, (1000, ))  # Random binary labels
X_test = np.random.randint(0, 10000, (200, 100))  # Random data for illustration
y_test = np.random.randint(0, 2, (200, ))  # Random binary labels

# Define your LSTM model architecture
model = Sequential([
    Embedding(input_dim=10000,  # Size of your vocabulary
              output_dim=128,  # Dimension of the dense embedding
              input_length=100),  # Length of input sequences
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

# Save the model for later use
model.save('path_to_my_model.h5')


2024-03-18 10:45:18.218023: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 10:45:18.452547: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 10:45:18.452641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 10:45:18.489650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 10:45:18.570062: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 10:45:18.573040: I tensorflow/core/platform/cpu_feature_guard.cc:1

Epoch 1/50
13/13 [==============================] - 3s 109ms/step - loss: 0.6932 - accuracy: 0.4875 - val_loss: 0.6930 - val_accuracy: 0.5050
Epoch 2/50
13/13 [==============================] - 1s 77ms/step - loss: 0.6741 - accuracy: 0.7700 - val_loss: 0.6933 - val_accuracy: 0.5100
Epoch 3/50
13/13 [==============================] - 1s 75ms/step - loss: 0.5685 - accuracy: 0.8512 - val_loss: 0.6952 - val_accuracy: 0.5700
Epoch 4/50
13/13 [==============================] - 1s 110ms/step - loss: 0.2505 - accuracy: 0.9850 - val_loss: 0.8074 - val_accuracy: 0.5400
Epoch 5/50
13/13 [==============================] - 1s 75ms/step - loss: 0.0603 - accuracy: 0.9975 - val_loss: 1.0763 - val_accuracy: 0.5250
Epoch 6/50
13/13 [==============================] - 1s 84ms/step - loss: 0.0221 - accuracy: 1.0000 - val_loss: 1.4370 - val_accuracy: 0.5500
Epoch 7/50
13/13 [==============================] - 1s 82ms/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 1.6529 - val_accuracy: 0.5350
Epoch 8/50


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
import numpy as np

# Placeholder: Load your data here

# Example placeholder data - replace with your actual data
X_train = np.random.randint(0, 10000, (1000, 100))  # Random data for illustration
y_train = np.random.randint(0, 2, (1000, ))  # Random binary labels
X_test = np.random.randint(0, 10000, (200, 100))  # Random data for illustration
y_test = np.random.randint(0, 2, (200, ))  # Random binary labels

# Define your LSTM model architecture
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2),  # Increased complexity and added dropout
    LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

# Using RMSprop optimizer and setting a learning rate
optimizer = RMSprop(lr=0.001)

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with early stopping
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

# Save the model for later use
model.save('path_to_my_model.h5')


Epoch 1/5
13/13 [==============================] - 7s 276ms/step - loss: 0.6934 - accuracy: 0.5013 - val_loss: 0.6922 - val_accuracy: 0.5250
Epoch 2/5
13/13 [==============================] - 3s 245ms/step - loss: 0.6917 - accuracy: 0.5275 - val_loss: 0.6922 - val_accuracy: 0.5250
Epoch 3/5
13/13 [==============================] - 3s 249ms/step - loss: 0.6908 - accuracy: 0.5350 - val_loss: 0.6922 - val_accuracy: 0.5250
Epoch 4/5
13/13 [==============================] - 3s 256ms/step - loss: 0.6903 - accuracy: 0.5275 - val_loss: 0.6922 - val_accuracy: 0.5250
Epoch 5/5
7/7 [==============================] - 0s 48ms/step - loss: 0.7161 - accuracy: 0.4750
Test Accuracy: 0.4749999940395355


In [3]:
from tensorflow.keras.models import load_model

model = load_model('path_to_my_model.h5')


In [ ]:
#Splitting the Data Frame Linux Terminal

In [33]:
# Split the CSV into 4 parts with a prefix 'data_chunk_'
# The -n l/4 option splits the file into 4 equal parts, by lines
#This will create files named data_chunk_00, data_chunk_01,

###   split -n l/10 -d your_large_file.csv data_chunk_


In [34]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("EmailClassification") \
    .getOrCreate()

# Replace the path below with the path to your Hadoop-stored file
df_emails = spark.read.csv("hdfs:///user1/data_batch_00", inferSchema=True, header=True)


In [38]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF

# Assuming the text column is named 'message'
# Tokenize the email content
tokenizer = RegexTokenizer(inputCol="message", outputCol="words", pattern="\\W")
tokenized_df = tokenizer.transform(df_emails)

# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filtered_df = remover.transform(tokenized_df)

# Continue with further processing as needed


In [46]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover

# Assuming 'df_emails' is your initial DataFrame and 'message' is the column with email texts
# Ensure 'df_emails' has been defined correctly and contains the 'message' column

# Tokenize the email content
tokenizer = RegexTokenizer(inputCol="message", outputCol="words", pattern="\\W")
tokenized_df = tokenizer.transform(df_emails)

# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filtered_df = remover.transform(tokenized_df)

# Check if the 'filtered_words' column exists
filtered_df.printSchema()

# If everything is correct up to here, then converting to a Pandas DataFrame should work
pandas_df = filtered_df.select("filtered_words").toPandas()


root
 |-- file: string (nullable = true)
 |-- message: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered_words: array (nullable = true)
 |    |-- element: string (containsNull = true)



24/03/20 10:09:40 ERROR Executor: Exception in task 1.0 in stage 30.0 (TID 51)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (RegexTokenizer$$Lambda$4435/158479843: (string) => array<string>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:217)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:388)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:888)
	at org.apache.spark.rdd.

Py4JJavaError: An error occurred while calling o712.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 30.0 failed 1 times, most recent failure: Lost task 1.0 in stage 30.0 (TID 51) (10.0.2.15 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (RegexTokenizer$$Lambda$4435/158479843: (string) => array<string>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:217)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apac

In [47]:
from pyspark.sql.functions import col
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover

# Filter out rows where the message column is null
df_non_null = df_emails.filter(col("message").isNotNull())

# Tokenize the email content
tokenizer = RegexTokenizer(inputCol="message", outputCol="words", pattern="\\W")
tokenized_df = tokenizer.transform(df_non_null)

# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filtered_df = remover.transform(tokenized_df)

# Continue with your processing...


In [48]:
# Convert to Pandas DataFrame
pandas_df = filtered_df.select("filtered_words").toPandas()

# Now, use Keras' text processing tools as shown previously to tokenize and pad sequences


In [49]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Placeholder for your data
texts = ["Your preprocessed email texts here..."]  # This should be a list of email texts
labels = [0, 1, 0, 1]  # Binary labels for each text

tokenizer = Tokenizer(num_words=10000)  # num_words is the size of your vocabulary
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences, maxlen=100)  # maxlen is the length of the longest sequence

# Assuming labels are already prepared
import numpy as np
labels = np.asarray(labels)


In [51]:
print(f"Data shape: {data.shape}")
print(f"Labels shape: {labels.shape}")


Data shape: (1, 100)
Labels shape: (4,)


In [52]:
import numpy as np

# Example placeholder, replace these with actual preprocessed data and labels
texts = ["sample text 1", "sample text 2", "sample text 3", "sample text 4"]
labels = [1, 0, 1, 0]

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences, maxlen=100)  # Ensure this matches your sequence length

labels = np.array(labels)

print(f"Data shape: {data.shape}")
print(f"Labels shape: {labels.shape}")

# Now proceed with train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)


Data shape: (4, 100)
Labels shape: (4,)


In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=100),  # input_dim is the size of the vocabulary, output_dim is the dimension of the dense embedding
    GlobalAveragePooling1D(),  # This will average the embeddings of all words in the sequence
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification (0 or 1 labels)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val), batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_2  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_6 (Dense)             (None, 24)                408       
                                                                 
 dense_7 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160433 (626.69 KB)
Trainable params: 160433 (626.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1/1 [==============================] - 1s 516ms/step - loss: 0.6947 -

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Define the LSTM model
model = Sequential([
    Embedding(input_dim=10000, output_dim=100, input_length=100),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val), batch_size=32)


Epoch 1/30
1/1 [==============================] - 2s 2s/step - loss: 0.6909 - accuracy: 0.6667 - val_loss: 0.7179 - val_accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 102ms/step - loss: 0.6823 - accuracy: 0.6667 - val_loss: 0.7376 - val_accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 121ms/step - loss: 0.6748 - accuracy: 0.6667 - val_loss: 0.7595 - val_accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - 0s 105ms/step - loss: 0.6666 - accuracy: 0.6667 - val_loss: 0.7815 - val_accuracy: 0.0000e+00
Epoch 5/30
1/1 [==============================] - 0s 94ms/step - loss: 0.6584 - accuracy: 0.6667 - val_loss: 0.8083 - val_accuracy: 0.0000e+00
Epoch 6/30
1/1 [==============================] - 0s 79ms/step - loss: 0.6504 - accuracy: 0.6667 - val_loss: 0.8437 - val_accuracy: 0.0000e+00
Epoch 7/30
1/1 [==============================] - 0s 99ms/step - loss: 0.6416 - accuracy: 0.6667 - val_loss: 0.8907 - val_accuracy: 0.0000e+0

Epoch 1/30
1/1 [==============================] - 3s 3s/step - loss: 0.6777 - accuracy: 0.6667 - val_loss: 0.7237 - val_accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 108ms/step - loss: 0.6848 - accuracy: 0.6667 - val_loss: 0.7403 - val_accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 120ms/step - loss: 0.6712 - accuracy: 0.6667 - val_loss: 0.7614 - val_accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - 0s 114ms/step - loss: 0.7053 - accuracy: 0.6667 - val_loss: 0.7783 - val_accuracy: 0.0000e+00
Epoch 5/30
1/1 [==============================] - 0s 111ms/step - loss: 0.6610 - accuracy: 0.6667 - val_loss: 0.7972 - val_accuracy: 0.0000e+00
Epoch 6/30
1/1 [==============================] - 0s 105ms/step - loss: 0.6236 - accuracy: 0.6667 - val_loss: 0.8238 - val_accuracy: 0.0000e+00


24/03/20 09:26:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- text: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered_words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)

Number of rows: 2
+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|               words|      filtered_words|        raw_features|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Another email con...|[another, email, ...|[another, email, ...|(6,[0,1,3,4],[1.0...|(6,[0,1,3,4],[0.0...|
|Email content exa...|[email, content, ...|[email, content, ...|(6,[0,1,2,5],[1.0...|(6,[0,1,2,5],[0.0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



+-------+--------------------+
|summary|                text|
+-------+--------------------+
|  count|                   2|
|   mean|                null|
| stddev|                null|
|    min|Another email con...|
|    max|Email content exa...|
+-------+--------------------+



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `message` cannot be resolved. Did you mean one of the following? [`text`].

In [59]:
def visualize_fraud_predictions(email_texts, predictions):
    """
    Visualize the emails with their predicted fraud probabilities.

    Parameters:
    - email_texts: List of email text content.
    - predictions: List of predicted probabilities corresponding to the fraud likelihood of each email.

    The function doesn't return anything but prints each email with its fraud prediction.
    """
    # Ensure the lists have the same length
    if len(email_texts) != len(predictions):
        print("Error: The length of email_texts and predictions must match.")
        return
    
    for email, probability in zip(email_texts, predictions):
        print("Email Content:\n", email)
        # Assuming the probability is given in a scale from 0 to 1
        print("Fraud Likelihood: {:.2%}".format(probability))
        print("-" * 100)

# Assuming the predictions list matches the number of email_texts provided
predicted_probabilities = [0.34725702, 0.08264993]

# Example usage with matching lengths for email_texts and predicted_probabilities
visualize_fraud_predictions(email_texts, predicted_probabilities)


NameError: name 'email_texts' is not defined

In [ ]:
def filter_emails_by_similarity_and_likelihood(emails, similarity_threshold=0.5, likelihood_threshold=8.0):
    """
    Filters emails based on content similarity to a given phrase and a likelihood threshold.
    
    Parameters:
    - emails: List of dictionaries, where each dictionary contains 'content' and 'likelihood' keys.
    - similarity_threshold: A threshold for determining content similarity (not used in this simple example).
    - likelihood_threshold: The minimum likelihood score for an email to be considered suspicious.
    
    Returns:
    - A list of emails considered suspicious based on the likelihood threshold.
    """
    suspicious_phrase = "Another suspicious email detected!"
    filtered_emails = [email for email in emails if suspicious_phrase in email['content'] and email['likelihood'] >= likelihood_threshold]
    return filtered_emails

# Example usage:
emails = [
    {'content': "This is a normal email content.", 'likelihood': 2.0},
    {'content': "Another suspicious email detected! Please check it out.", 'likelihood': 8.26},
    {'content': "Another suspicious email detected! This seems like a scam.", 'likelihood': 9.5},
    {'content': "This is another normal conversation.", 'likelihood': 3.2}
]

# Filtering emails:
suspicious_emails = filter_emails_by_similarity_and_likelihood(emails, likelihood_threshold=8.0)

# Displaying the filtered, suspicious emails:
for email in suspicious_emails:
    print(f"Email Content: {email['content']}")
    print(f"Fraud Likelihood: {email['likelihood']}%")
    print("-"*80)
